# Part 1 - Quickstart Using Docker Compose to Locally Run PyGrid

In this tutorial, you'll learn how to use Docker to deploy a PyGrid Gateway and 4 Nodes into a local machine and then interact with it using PySyft. This quickly gets you up and running to test out the functionality of PySyft with the additonal management power of PyGrid. 

This method would not be used to setup a production environment. Instead, you would deploy the Gateway to a cloud server, and each Node to a separate client (ie. phone, hospital, etc). See the next tutorial for that: Part 2 - Deploying PyGrid Using the CLI.

_WARNING: Grid nodes publish datasets online and are for EXPERIMENTAL use only. Deploy nodes at your own risk. Do not use OpenGrid with any data/models you wish to keep private._


### Step 1: Download the repository

```bash
git clone https://github.com/OpenMined/PyGrid/
```

### Step 2: Download dependencies

You'll need to have the app dependencies installed. Use Python 3.6 or 3.7. We recommend setting up an independent [conda environment](https://docs.conda.io/projects/conda/en/latest/user-guide/concepts/environments.html) to avoid problems with library versions.

You can install the dependencies by running:

```bash
cd PyGrid
pip install -r requirements.txt
```

### Step 3: Start Docker Containers

First, [install Docker](https://docs.docker.com/get-docker/) if it isn't already installed on your computer.

This tutorial is composed of 6 Docker containers: 1 Grid Gateway, 1 Redis database, and 4 Grid Nodes. You can think of it like each service could be run on a different computer. Technically the Redis container would be run on each computer that hosts a Grid Node. For the ease of this tutorial, we'll run all the services on your computer over localhost.
    
To easily get all these services up, we use Docker Compose. From the Docker documentation:

> Compose is a tool for defining and running multi-container Docker applications. With Compose, you use a YAML file to configure your application’s services. Then, with a single command, you create and start all the services from your configuration.

Powerful stuff. The file `PyGrid/docker-compose.yml` specifies the configuration. If you're unclear on what Docker Compose does, give this file a look and it should be more clear what's going on.

To actually start the Docker containers, run the following in the same directory as `docker-compose.yml`:

```bash
docker-compose up
```

### Step 4: Creating a Grid Worker and start communication

First, import the required libraries. Then, hook PyTorch with PySyft to extend the functionality of PyTorch.

In [1]:
import torch as th
import syft as sy
from syft.workers.node_client import NodeClient
from syft.grid.public_grid import PublicGridNetwork

hook = sy.TorchHook(th)

Connect to a Node worker called Bob with WebSockets. In a production environment, this could be a phone, hospital, etc. 

***WARNING:*** Make sure to use the same address/port as the one used to start the worker! The code as it is will work if you're using docker-compose.

In [2]:
bob = NodeClient(hook, "ws://localhost:3000")

bob.id

'Bob'

Initialize a Torch tensor with toy data. Then, send the data to Bob and get back a pointer to x, which points to the data that is now on Bob.

In [3]:
data = th.tensor([1,2,3,4])
x = data.send(bob)

x

(Wrapper)>[PointerTensor | me:61027395165 -> Bob:11556638281]

We're finally at the heart of what makes PySyft useful: computation. We're going to perform a remote computation on the data we sent to Bob, without having the data in this notebook. 

Note that we're going to see a pointer tensor instead of a numeric result. This is expected. We'll get the result back in the next step. You can imagine that many more computations would typically be run on the data before we get back the final result.

In [4]:
y = x + x

y

(Wrapper)>[PointerTensor | me:39529581238 -> Bob:63140690368]

Get the numeric result of the remote computation from Bob.

In [5]:
y = y.get()

y

tensor([2, 4, 6, 8])

### Step 5: Start communication with the Grid Gateway and workers

Connect to the Grid Gateway running on Docker. It allows us to interact with the Node workers we also started with Docker.

In [6]:
gateway = PublicGridNetwork(hook,"http://localhost:5000")

Connect to the worker nodes.

***WARNING:*** Make sure to use the same address/port as the one used to start the worker! The code as it is will work if you're using docker-compose.

In [7]:
bob = NodeClient(hook, "ws://localhost:3000")
alice = NodeClient(hook, "ws://localhost:3001")

Send some toy data to Bob, and tag it with #tensor to be searchable.

In [8]:
x = th.tensor([1, 2, 3, 4, 5]).tag("#tensor").send(bob)

x

(Wrapper)>[PointerTensor | me:16721743307 -> Bob:48327504592]
	Tags: #tensor 
	Shape: torch.Size([5])

Now, the gateway searches for any datasets tagged #tensor across ALL connected workers. It should show the dataset we just sent to Bob.

In [9]:
gateway.search("#tensor")

{'Bob': [(Wrapper)>[PointerTensor | me:37705196978 -> Bob:48327504592]
  	Tags: #tensor 
  	Shape: torch.Size([5])]}

Also send some toy data to Alice, and tag it with #tensor to be searchable.

In [10]:
y = th.tensor([5, 4, 3, 2, 1]).tag("#tensor").send(alice)

y

(Wrapper)>[PointerTensor | me:61000211801 -> Alice:52617331829]
	Tags: #tensor 
	Shape: torch.Size([5])

The gateway should now show the datasets we sent to Bob and to Alice.

In [11]:
z = gateway.search("#tensor")

z

{'Bob': [(Wrapper)>[PointerTensor | me:69104242676 -> Bob:48327504592]
  	Tags: #tensor 
  	Shape: torch.Size([5])],
 'Alice': [(Wrapper)>[PointerTensor | me:3231242668 -> Alice:52617331829]
  	Tags: #tensor 
  	Shape: torch.Size([5])]}

### Next Steps

We can use the gateway to do all kind of jobs. We saw it already search for datasets on workers by tags.

We can also use the gateway to do things like querying workers, serving models (either encrypted or unencrypted), and running remote inferences.

In the next tutorial, we'll see how we can launch these services one-at-a-time, instead of all-at-once with Docker Compose. This will get us towards being able to deploy a gateway and nodes for an actual application!